In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
# from sentence_transformers import SentenceTransformer
import os 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import FAISS
model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', 
                                            model_kwargs={'device':'cuda'})
# model = sentence_transformer( model_name ='sentence-transformers/all-MiniLM-L6-v2')
sample_encoding = model.embed_query("Sample Sentence")
print("Dimension of the Embedding : ", len(sample_encoding))

c:\Users\krish\anaconda3\envs\torch_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dimension of the Embedding :  384


In [2]:
openai_embedding_model = OpenAIEmbeddings(model='text-embedding-ada-002',
                                                       openai_api_key=os.getenv('OPENAI_KEY'))
sample_encoding = openai_embedding_model.embed_query("Sample Sentence")
print("Dimension of the Embedding : ", len(sample_encoding))

Dimension of the Embedding :  1536


In [13]:
import os
import pandas as pd
from dateutil.parser import parse
df = pd.read_csv('.\\Canon Open Source Data\\excel knowledge base.txt', delimiter='\t')
df.drop([0], axis=0, inplace=True)
df.drop('Field name', axis =1 , inplace=True)
df.rename(columns={'Record No.':'index'}, inplace=True)
df.drop('index', axis=1, inplace =True)
df.fillna("Not Available", inplace =True)
df['Date'] = [parse(df.loc[i].Date) for i in df.index.to_list()]

In [14]:
df

,Date,Department,Product,Zone,State,Category,Competetor_Product_Information,Attachment
1,2023-06-23,PCC-BIS,CS,India,#VALUE!,Economic,Due to Growth in Road Logistics Industry New O...,Not Available
2,2023-06-23,PCC-BIS,CS,West,#VALUE!,Product,Sharp organized partners meet synergy at Goa i...,Not Available
3,2023-06-30,PCC-PPP,PPP,India,#VALUE!,Product,HP is very aggressively positioning Z6 series ...,Not Available
4,2023-06-29,PBC,ICB,East,#VALUE!,Product,Nikon & Sony started promoting Z8 and Zve1 vis...,Not Available
5,2023-06-22,CBC-PPP,PPP,India,#VALUE!,Market footfall,Konica Minolta demo on wheels had good respons...,Not Available
...,...,...,...,...,...,...,...,...
83,2023-07-14,PBC-South,ICB,South,#VALUE!,Product,Sony A7M4 sellout is good in the market.,Not Available
84,2023-07-14,PBC-South,ICB,South,#VALUE!,Product,Sony New model A7R5 stocks reached outlets las...,Not Available
85,2023-07-14,PBC-South,ICB,South,#VALUE!,Product,Sony APS-C stocks likely to hit market post 15...,Not Available
86,2023-07-14,PBC-South,ICB,South,#VALUE!,Product,Nikon Z8 stocks are available in market but no...,Not Available


In [16]:
df.to_csv('.\\Canon Open Source Data\\knowledgeBase.csv')

In [17]:
import mysql.connector as connection 

try:
    mydb=connection.connect(host="localhost",user="root",password='1320')
    print(mydb.is_connected())                                   
    #mydb.close()
except Exception as e:
    print(str(e))

True


In [3]:
# creating a new database 

query= "create database CanonDB"
cursor=mydb.cursor()           
cursor.execute(query)          
print("Database created !")

Database created !


In [18]:
try:
    mydb=connection.connect(host="localhost",database="canonDB",user="root",password="1320",use_pure=True)
    print(mydb.is_connected())                                   # To check if connection is established with the Database
    #mydb.close()
except Exception as e:
    print(str(e))

True


In [8]:
db=connection.connect(host="localhost",database='canonDB',user="root",password="1320",use_pure=True)
cursor=db.cursor()
query = "Drop table canon"
cursor.execute(query)
db.commit()

In [19]:
# Step 4 : Inserting data into the Table

db=connection.connect(host="localhost",database='canonDB',user="root",password="1320",use_pure=True)
cursor=db.cursor()

# cols = "`,`".join(['ID']+[str(i) for i in df.columns.tolist()])
colvalues=[]
for i,row in df.iterrows():
    colvalues.append(tuple([i+1]+list(row)))

# Insert DataFrame recrds one by one.
for i in colvalues:
    try:
        query = "INSERT INTO CANON VALUES{}".format(i)
        cursor.execute(query)
        db.commit()
    except Exception as e:
        print(e)

In [20]:
# Step5 : Retreiving the same data from the MYSQLdatabase 
db=connection.connect(host="localhost",database='canonDB',user="root",password="1320",use_pure=True)
query="Select * from Canon"
df=pd.read_sql(query,db)
df.head(3)

C:\Users\krish\AppData\Local\Temp\ipykernel_24580\3438997778.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(query,db)


,index,Date,Department,Product,Zone,State,Category,Competetor_Product_Information,Attachment
0,2,2023-06-23,PCC-BIS,CS,India,#VALUE!,Economic,Due to Growth in Road Logistics Industry New O...,Not Available
1,3,2023-06-23,PCC-BIS,CS,West,#VALUE!,Product,Sharp organized partners meet synergy at Goa i...,Not Available
2,4,2023-06-30,PCC-PPP,PPP,India,#VALUE!,Product,HP is very aggressively positioning Z6 series ...,Not Available


In [21]:
df.dtypes

index                              int64
Date                              object
Department                        object
Product                           object
Zone                              object
State                             object
Category                          object
Competetor_Product_Information    object
Attachment                        object
dtype: object

In [1]:
db=connection.connect(host="localhost",database='canonDB',user="root",password="1320",use_pure=True)
query="drop table canon"
df=pd.read_sql(query,db)
df.head(3)

In [5]:
current_path = os.getcwd()
media_path = 
os.rmdir('media')

OSError: [WinError 145] The directory is not empty: 'media'

In [3]:
%pwd

'c:\\Users\\krish\\OneDrive\\Desktop\\Study\\Gen AI\\Multi-PDF-QnA-Bot'

In [2]:
import pandas as pd 
from dateutil.parser import parse
df = pd.read_csv('.\\Canon Open Source Data\\knowledgeBase.csv')
df['Date'] = [parse(df.loc[i].Date) for i in range(df.shape[0])]
# df.drop('Unnamed: 0', axis=1, inplace=True)
df

,index,Date,Department,Product,Zone,State,Category,Competetor_Product_Information,Attachment
0,0,2023-06-23,PCC-BIS,CS,India,#VALUE!,Economic,Due to Growth in Road Logistics Industry New O...,Not Available
1,1,2023-06-23,PCC-BIS,CS,West,#VALUE!,Product,Sharp organized partners meet synergy at Goa i...,Not Available
2,2,2023-06-30,PCC-PPP,PPP,India,#VALUE!,Product,HP is very aggressively positioning Z6 series ...,Not Available
3,3,2023-06-29,PBC,ICB,East,#VALUE!,Product,Nikon & Sony started promoting Z8 and Zve1 vis...,Not Available
4,4,2023-06-22,CBC-PPP,PPP,India,#VALUE!,Market footfall,Konica Minolta demo on wheels had good respons...,Not Available
...,...,...,...,...,...,...,...,...,...
82,82,2023-07-14,PBC-South,ICB,South,#VALUE!,Product,Sony A7M4 sellout is good in the market.,Not Available
83,83,2023-07-14,PBC-South,ICB,South,#VALUE!,Product,Sony New model A7R5 stocks reached outlets las...,Not Available
84,84,2023-07-14,PBC-South,ICB,South,#VALUE!,Product,Sony APS-C stocks likely to hit market post 15...,Not Available
85,85,2023-07-14,PBC-South,ICB,South,#VALUE!,Product,Nikon Z8 stocks are available in market but no...,Not Available


In [9]:
df.Product.value_counts()

ICB      37
PPP      24
CS       23
Laser     3
Name: Product, dtype: int64

In [12]:
df.loc[df.Date=="30-Jun-23"].Zone.value_counts()

North    12
India     3
South     2
Name: Zone, dtype: int64

In [3]:
import os
import pandas as pd
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.agents import create_sql_agent
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

In [6]:
class SQLQuery:
    def __init__(self):
        self.DB_USERNAME = os.getenv('DB_USERNAME')
        self.DB_PASSWORD = os.getenv('DB_PASSWORD')
        self.DB_HOST = os.getenv('DB_HOST')
        self.DB_NAME = os.getenv('DB_NAME')

    def createDBConnectionString(self):
        db_user = self.DB_USERNAME
        db_Password  = self.DB_PASSWORD
        db_host = self.DB_HOST
        db_name = self.DB_NAME
        connectionString = f"mysql+pymysql://{db_user}:{db_Password}@{db_host}/{db_name}"
        return connectionString
    
    def getSQLSchema(self):
        sql_query = f"""  
        SELECT C.TABLE_NAME, C.COLUMN_NAME, C.DATA_TYPE, T.TABLE_TYPE, T.TABLE_SCHEMA  
        FROM INFORMATION_SCHEMA.COLUMNS C  
        JOIN INFORMATION_SCHEMA.TABLES T ON C.TABLE_NAME = T.TABLE_NAME AND C.TABLE_SCHEMA = T.TABLE_SCHEMA  
        WHERE T.TABLE_SCHEMA = '{self.DB_NAME}' 
        """ 
        mysql_connection_string = self.createDBConnectionString()
        result = pd.read_sql_query(sql_query, mysql_connection_string)
        df = result.infer_objects()
        output=[]
        current_table = ''  
        columns = []  
        for index, row in df.iterrows():
            table_name = f"{row['TABLE_SCHEMA']}.{row['TABLE_NAME']}"  
            column_name = row['COLUMN_NAME']  
            data_type = row['DATA_TYPE']  
            if " " in table_name:
                table_name= f"[{table_name}]" 
            column_name = row['COLUMN_NAME']  
            if " " in column_name:
                column_name= f"[{column_name}]" 
                # If the table name has changed, output the previous table's information  
            if current_table != table_name and current_table != '':  
                output.append(f"table: {current_table}, columns: {', '.join(columns)}")  
                columns = []  
            
            # Add the current column information to the list of columns for the current table  
            columns.append(f"{column_name} {data_type}")  
            
            # Update the current table name  
            current_table = table_name  

        # Output the last table's information  
        output.append(f"table: {current_table}, columns: {', '.join(columns)}")
        output = "\n ".join(output)

        return output

In [7]:
obj = SQLQuery()
conn_string = obj.createDBConnectionString()
df = obj.getSQLSchema()
df

'table: canondb.canon, columns: index int, Date date, Department varchar, Product varchar, Zone varchar, State varchar, Category varchar, Competetor_Product_Information varchar, Attachment varchar'

In [12]:
import os 
import pinecone as pc
from dotenv import load_dotenv
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import FAISS, Pinecone

load_dotenv()

class Data:
    def __init__(self, pdf_data_path, vector_db_path):
        self.pdf_data_path = pdf_data_path
        self.vector_db_path = vector_db_path
        self.pinecone_api_key = os.getenv('PINECONE_KEY')
        self.pinecone_env = os.getenv('PINECONE_ENV')
        self.embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

    def createPDFVectorDBwithFAISS(self):
        loader = DirectoryLoader(self.pdf_data_path, glob='*.pdf', loader_cls=PyPDFLoader,
                                use_multithreading=True, show_progress=True)
        docs = loader.load()
        text_splitter  = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200, 
                                                        separators=["\n\n", "\n", " ", ""])
        texts = text_splitter.split_documents(docs)
        db = FAISS.from_documents(texts, self.embedding_model)
        db.save_local(self.vector_db_path)

    def create_top_k_chunk_from_FAISS(self, question, top_k):
        test_idex = FAISS.load_local(self.vector_db_path,self.embedding_model)
        top_k_chunks  = test_idex.similarity_search(question,k=top_k)
        return top_k_chunks
    

    def createPDFVectorDBwithPinecone(self):
        loader = DirectoryLoader(self.pdf_data_path, glob='*.pdf', loader_cls=PyPDFLoader,
                                use_multithreading=True, show_progress=True)
        docs = loader.load()
        text_splitter  = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200, 
                                                        separators=["\n\n", "\n", " ", ""])
        texts = text_splitter.split_documents(docs)
        embeddings = []
        ids = []
        metadatas = []
        for i in range(len(texts)):
            page_content = texts[i].page_content
            source_pdf = texts[i].metadata['source'].split('\\')[-1]
            page_number = str(texts[i].metadata['page'])
            embedded_page_content = self.embedding_model.embed_query(page_content)
            metadata = {
                'source' : source_pdf,
                'page' : page_number,
                'page_content' : page_content
            }
            ids.append(str(i))
            embeddings.append(embedded_page_content)
            metadatas.append(metadata)

        pc.init(api_key=self.pinecone_api_key, environment=self.pinecone_env)
        index = pc.Index('pdf-index')
        index.upsert(vectors = zip(ids, embeddings, metadatas))


    def create_top_k_chunk_from_Pinecone(self, question, top_k):
        pc.init(api_key=self.pinecone_api_key, environment=self.pinecone_env)
        index = pc.Index('pdf-index')
        #embedded_question = self.embedding_model.embed_query(question)
        vectorstore = Pinecone( index, self.embedding_model.embed_query, text_key='page_content')
        top_k_chunks = vectorstore.similarity_search(question, k=top_k)
        #result = index.query(embedded_question, top_k = top_k, includeMetaData=True)
        return top_k_chunks

In [13]:
pdf_data_path = ".\\media"
pdf_vector_embedding_path = ".\\VectorDB"
data_obj = Data(pdf_data_path,pdf_vector_embedding_path)
#data_obj.createPDFVectorDBwithPinecone()
test_question = "Find the cost of Sony ZV-E1 Full Frame camera"
result = data_obj.create_top_k_chunk_from_Pinecone(test_question, top_k =3)
print(result[0].metadata['source'])
print(result[0].metadata['page'])

Sony ZV-E1 Full-Frame Interchangeable-Lens Mirrorless vlog Camera.pdf
1


In [14]:
top_k_chunks = result

In [17]:
def createQuestionPrompt(question, top_k_chunks):
        prompt= ""
        prompt += 'search results:\n\n'
        for i in range(len(top_k_chunks)):
            meta_info = '['+top_k_chunks[i].metadata['source'].split('\\')[-1] + " , page : " + str(top_k_chunks[i].metadata['page'])+'] '
            page_content = top_k_chunks[i].page_content.replace('\n', ' ')
            page_content = re.sub('\s+', ' ', page_content)
            combined_content = meta_info + page_content
            prompt += combined_content +'\n\n'
        prompt += "Instructions: Compose a comprehensive reply to the query using the search results given."\
                    "Cite each reference using [pdfname.pdf , page : number] notation (every result has this number at the beginning)."\
                    "Citation should be done at the end of each sentence. If the search results mention multiple subjects"\
                    "with the same name, create separate answers for each. Only include information found in the results and"\
                    "don't add any additional information. Make sure the answer is correct and don't output false content."\
                    "If the text does not relate to the query, simply state 'Found Nothing'. Don't write 'Answer:'"\
                    "Directly start the answer.\n"
        prompt+= f"Query : {question} \n\n"
        return prompt

In [18]:
question = "Find the cost of Sony ZV-E1 Full Frame camera"
createQuestionPrompt(question, top_k_chunks)

'search results:\n\n[Sony ZV-E1 Full-Frame Interchangeable-Lens Mirrorless vlog Camera.pdf , page : 1] Technical Details: Brand Sony Manufacturer Sony, Sony Corporation, 1 -7-1 Konan, Minato -KU, Tokyo 108 -0075, Japan Model ZV-E1/BQ IN5 Model Name ZV-E1 Model Year 2023 Product Dimensions 5.43 x 12.1 x 7.19 cm; 483 Grams Batteries 1 Lithium Ion batteries required. (included) Item model number ZV-E1/BQ IN5 Memory Slots Available 1 Compatible Devices Laptop, Desktop, Tablet, Smartphone Special Features Face Detection Mounting Hardware Camera, Rechargeable Battery NP -FZ100, Shoulder strap, Wind Screen, Wind Screen Adapter, Body cap, Accessory shoe cap Number of items 1 Standing screen display size\n\n[Sony ZV-E1 Full-Frame Interchangeable-Lens Mirrorless vlog Camera.pdf , page : 0] Sony ZV -E1 Full -Frame Interchangeable -Lens Mirrorless vlog Camera Working with the ZV -E1\'s 12.1 MP2 full -frame sensor, the latest BIONZ XR ™ engine provides high - speed processing to support 4K 60p, 120